In [1]:
import pandas as pd
import numpy as np
import setting

benchmark_daily=pd.read_csv(setting.datapath+'bab_market_excess.csv')

benchmark_daily=benchmark_daily[['Date','Return']]

benchmark_monthly=pd.read_csv(setting.datapath+'bab_market_excess_monthly.csv')
benchmark_monthly.columns=['Date','Return']
        
    

In [2]:
aapl=pd.read_csv(setting.datapath+'aapl.csv')
aapl.columns=['PERMNO','Date','Return']
aapl['Date']=pd.to_datetime(aapl['Date'],format='%d/%m/%Y')
aapl=aapl[['Date','Return']]
aapl=aapl.set_index('Date')

In [3]:
benchmark_monthly['Date']=pd.to_datetime(benchmark_monthly['Date'],format='%d/%m/%Y')
benchmark_daily['Date']=pd.to_datetime(benchmark_daily['Date'],format='%d/%m/%Y')

In [4]:
benchmark_monthly=benchmark_monthly.set_index('Date')
benchmark_daily=benchmark_daily.set_index('Date')
benchmark_daily['vol']=0

In [5]:
#Calculate the benchmark volatility directly
for i in range(120,len(benchmark_daily.index.values)):
    benchmark_daily['vol'].iloc[i]=benchmark_daily['Return'].iloc[i-120:i].std()

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
def calculate_beta(stock,monthly=False,log_return=False):
    '''
    The function use stock and benchmark return series to calculate the beta using the methodology
    intriduced in tFrazzini's paper "Betting Against Beta"
    
    Input:
    stock: pandas DataFrame of certain stock return with date as index
    
    benchmark: pandas DataFrame of market benchmark return with date as index
    monthly: indicate if the return series is monthly, if so, a different calculation methoh will be used
    log_return: indicate of the return is caluclated as log return, if not, will convert
    '''
    if log_return==False:
        #Convert arithmatic return into log-return
        stock['Return']=np.log(stock['Return']+1)
    if monthly:
        benchmark=benchmark_monthly
    else:
        benchmark=benchmark_daily
    stock_dates=pd.Series(stock.index.values,index=stock.index)
    
    
    stock_month_end=stock_dates.resample('M').max()
   
    
    #Merge the market return with the stock return data:
    market=pd.concat([stock,benchmark],axis=1)
   
    market.columns=['stock','mkt','mkt vol']
    #calculate overlapping three days log return
    market['stock overlap']=market['stock']+market['stock'].shift(1)+market['stock'].shift(2)
    market['mkt overlap']=market['mkt']+market['mkt'].shift(1)+market['mkt'].shift(2)
    market['stock vol']=0
    market['corr']=0
    #Calculate volatility using 120 days data
    target_date=list(stock_month_end)
    
    
    beginning_position=list(market.index.values).index(min(stock.index.values))
    target_date = np.array(target_date, dtype=np.datetime64)
    for i in target_date:
        position=list(market.index.values).index(i)
        if position>=beginning_position+120:
            #print(position)
            market['stock vol'].iloc[position]=market['stock'].iloc[position-120:position].std()
            #market['mkt vol'].iloc[position]=market['mkt'].iloc[position-119:position].std()
       
    #calculate correlation
    for i in target_date:
        
        position=list(market.index.values).index(i)
        if position>=beginning_position+750:
            correlation=market[['stock overlap','mkt overlap']].iloc[position-750:position]
            market['corr'].iloc[position]=np.correlate(correlation['stock overlap'],correlation['mkt overlap'])

    market['beta']=market['corr']*market['stock vol']/market['mkt vol']
    
    #Adjustment on beta
    result=market.iloc[beginning_position+750:]
    result=result['beta'][result.index.isin(target_date)]
    result=0.6*result+0.4
    
    
    return result

In [7]:
result=calculate_beta(aapl)

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
result

In [8]:
stock = aapl

In [11]:
stock_dates=pd.Series(stock.index.values,index=stock.index)

In [13]:
stock_month_end=stock_dates.resample('M').last()

In [10]:
benchmark=benchmark_daily

In [15]:
#Merge the market return with the stock return data:
market=pd.concat([stock,benchmark],axis=1)

market.columns=['stock','mkt','mkt vol']
#calculate overlapping three days log return
market['stock overlap']=market['stock']+market['stock'].shift(1)+market['stock'].shift(2)
market['mkt overlap']=market['mkt']+market['mkt'].shift(1)+market['mkt'].shift(2)
market['stock vol']=0
market['corr']=0

In [16]:

#Calculate volatility using 120 days data
target_date=list(stock_month_end)

In [18]:
beginning_position=list(market.index.values).index(min(stock.index.values))

In [28]:
import timeit

In [29]:
timeit.timeit('market['stock vol'] = market['stock'].rolling(120).std()')


SyntaxError: invalid syntax (<ipython-input-29-fbd75a26478e>, line 1)

In [25]:
market

,stock,mkt,mkt vol,stock overlap,mkt overlap,stock vol,corr
Date,,,,,,,
1963-01-02,NaN,-0.007509,0.000000,NaN,NaN,NaN,0
1963-01-03,NaN,0.013969,0.000000,NaN,NaN,NaN,0
1963-01-04,NaN,0.004396,0.000000,NaN,0.010856,NaN,0
1963-01-07,NaN,-0.001554,0.000000,NaN,0.016811,NaN,0
1963-01-08,NaN,0.006531,0.000000,NaN,0.009373,NaN,0
1963-01-09,NaN,-0.003972,0.000000,NaN,0.001005,NaN,0
1963-01-10,NaN,-0.000517,0.000000,NaN,0.002042,NaN,0
1963-01-11,NaN,0.000111,0.000000,NaN,-0.004378,NaN,0
1963-01-14,NaN,0.003232,0.000000,NaN,0.002826,NaN,0


In [ ]:
target_date = np.array(target_date, dtype=np.datetime64)
for i in target_date:
    position=list(market.index.values).index(i)
    if position>=beginning_position+120:
        #print(position)
        market['stock vol'].iloc[position]=market['stock'].iloc[position-120:position].std()
        #market['mkt vol'].iloc[position]=market['mkt'].iloc[position-119:position].std()

#calculate correlation
for i in target_date:

    position=list(market.index.values).index(i)
    if position>=beginning_position+750:
        correlation=market[['stock overlap','mkt overlap']].iloc[position-750:position]
        market['corr'].iloc[position]=np.correlate(correlation['stock overlap'],correlation['mkt overlap'])

market['beta']=market['corr']*market['stock vol']/market['mkt vol']

#Adjustment on beta
result=market.iloc[beginning_position+750:]
result=result['beta'][result.index.isin(target_date)]
result=0.6*result+0.4

